### 3. Naiver Bayesklassifikator zur Gesichtserkennung

Implementieren Sie den Gaussian-Naïve-Bayes-Klassifikator aus der Vorlesung. Testen Sie Ihre Implementierung am Datensatz ''Labeled Faces in the Wild'' aus Aufgabe 2, wiederum nur für Personen, für die mindestens 70 Bilder existieren. Teilen Sie Ihren Datensatz in 60 % Trainings- und 40% Testdaten (nach vorheriger Zufalls-Permutation der Reihenfolge) und skalieren Sie die Bilder wieder auf 1/8 der Originalgröße. Führen Sie anschließend eine Hauptkomponentenanalyse auf den Trainingsdaten durch und projizieren Sie sowohl Trainings- als auch Testbilder auf die ersten 7 Eigengesichter. Trainieren Sie Ihren GNB-Klassifikator auf dem Trainingsdatensatz als ''George-W.-Bush-Detektor'', d.h. alle zu dieser Person gehörigen Bilder werden mit 1 gelabelt, alle sonstigen mit –1. Werten Sie Ihren Klassifikator sowohl auf den Trainings- wie auf den unabhängigen Testdaten aus. Bestimmen Sie dafür jeweils die Detektionswahrscheinlichkeit, Richtig-Negativ-Rate, Fehlalarmrate und Falsch-Negativ-Rate.


In [1]:
import numpy as np

from pca import PCA
from naive_bayes import GaussianNaiveBayes
from face_vectorizer import FaceVectorizer

In [14]:
train_images, test_images, train_labels, test_labels = FaceVectorizer(
    "./data/faces_in_the_wild/lfw_funneled/").get_images()

In [15]:
pca = PCA()
pca.fit(train_images)

eigenfaces = pca.v @ pca.s

In [16]:
test_projections = test_images @ eigenfaces[0:7, :].T
train_projections = train_images @ eigenfaces[0:7, :].T

In [17]:
train_labels[train_labels != "George_W_Bush"] = -1
train_labels[train_labels == "George_W_Bush"] = 1

test_labels[test_labels != "George_W_Bush"] = -1
test_labels[test_labels == "George_W_Bush"] = 1
# Cast labels from Unicode-String to int8
train_labels = train_labels.astype("int8")
test_labels = test_labels.astype("int8")
# Flatten labels
train_labels = train_labels.flatten()
test_labels = test_labels.flatten()

In [46]:
nb = GaussianNaiveBayes()
nb.fit(train_projections, train_labels)

### Trainset

In [49]:
y_hat = nb.predict(train_projections)

In [51]:
# Sensitivität
np.count_nonzero((train_labels == 1) & (y_hat == 1)) / len(y_hat)

0.26307572209211555

In [53]:
# Specificity
np.count_nonzero((train_labels == -1) & (y_hat == -1)) / len(y_hat)

0.34192037470725994

In [54]:
# False-Positive
np.count_nonzero((train_labels == -1) & (y_hat == 1)) / len(y_hat)

0.24512099921935987

In [56]:
# False-Negative
np.count_nonzero((train_labels == 1) & (y_hat == -1)) / len(y_hat)

0.14988290398126464

### Testset

In [6]:
y_hat = nb.predict(test_projections)

In [7]:
y_hat

array([-1., -1.,  1.,  1., -1.,  1., -1.])

In [8]:
# Validierung mit Sklearn Implementierung

from sklearn.naive_bayes import  GaussianNB

sk_nb = GaussianNB().fit(train_projections, train_labels.flatten())
sk_nb.predict(test_projections)

array([-1, -1,  1,  1, -1,  1, -1], dtype=int8)

In [34]:
# Sensitivität
np.count_nonzero((test_labels == 1) & (y_hat == 1)) / len(y_hat)

0.14285714285714285

In [35]:
# Specificity
np.count_nonzero((test_labels == -1) & (y_hat == -1)) / len(y_hat)

0.5714285714285714

In [42]:
# False-Positive
np.count_nonzero((test_labels == -1) & (y_hat == 1)) / len(y_hat)

0.2857142857142857

In [43]:
# False-Negative
np.count_nonzero((test_labels == 1) & (y_hat == -1)) / len(y_hat)

0.0